---
format:
  html:
    embed-resources: true
---

# Exploratory Data Analysis of TV Sales Across Major Retailers

### Introduction:

iDC TV Inc. aims to regain its market presence by understanding and countering the promotional strategies and pricing structures of its competitors. By leveraging data from major competitors, the objective is to derive insights that can aid in formulating an effective pricing strategy.

### Section 1: Market Overview

###  1.1 Market Presence and Customer Preference:

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Load the datasets
file_paths = {
    'Walmart': 'walmart_data_cleaned.csv',
    'Costco': 'costco_data_cleaned.csv',
    'BestBuy': 'bestbuy_data_cleaned.csv',
    'Amazon': 'amazon_data_cleaned.csv'
}

datasets = {}
for retailer, path in file_paths.items():
    datasets[retailer] = pd.read_csv(path)

all_data = []
for retailer, df in datasets.items():
    df = df.copy()
    df['Retailer'] = retailer
    all_data.append(df)

all_df = pd.concat(all_data, axis=0, ignore_index=True)

In [2]:
aggregated_data = []
for retailer, df in datasets.items():
    grouped = df.groupby('Brand')['Review Number'].sum().reset_index()
    grouped['Retailer'] = retailer
    aggregated_data.append(grouped)

# Concatenate the aggregated data
aggregated_df = pd.concat(aggregated_data, axis=0, ignore_index=True)

top_brands = aggregated_df.groupby('Brand')['Review Number'].sum().reset_index()
top_brands = top_brands.sort_values(by='Review Number', ascending=False).head(10)['Brand'].tolist()

filtered_df = aggregated_df[aggregated_df['Brand'].isin(top_brands)]

fig = px.bar(
    filtered_df, x='Brand', y='Review Number', color='Retailer', text='Review Number',
    title='Top 10 Brands based on Number of Reviews across Different Retailers',
    labels={'Review Number': 'Number of Reviews', 'Brand': 'TV Brand'},
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_layout(showlegend=True, xaxis_title='TV Brand', yaxis_title='Number of Reviews',
                  xaxis={'categoryorder': 'total descending'},
                  uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.show()

In [3]:
go.Figure(fig).write_html("eda_plots/iDC_plot1.html", auto_open=True)

In [4]:
filtered_df = aggregated_df[aggregated_df['Brand'].isin(top_brands)]

pie_data = filtered_df.groupby('Brand')['Review Number'].sum().reset_index()

pie_data = pie_data.sort_values(by='Review Number', ascending=False)

fig = px.pie(
    pie_data, names='Brand', values='Review Number',
    title='Distribution of Reviews among Top 10 Brands',
    color='Brand', color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_traces(textinfo='percent+label', pull=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
fig.update_layout(showlegend=True)

fig.show()

In [5]:
go.Figure(fig).write_html("eda_plots/iDC_plot2.html", auto_open=True)

Samsung is the top seller, with the most reviews, followed by Vizio in second place. Together, they hold more than 55% of the market, showing their strong influence on customer choices and market trends.

#### 1.2 Retailer Performance Analysis:

In [6]:
df = all_df.groupby('Retailer')['Review Number'].sum().reset_index()

fig = px.bar(
    df, x='Retailer', y='Review Number', color='Retailer', text='Review Number',
    title='Total Number of Reviews for each Retailer',
    labels={'Review Number': 'Number of Reviews', 'Retailer': 'Retailer'},
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_layout(
    showlegend=True, 
    xaxis_title='Retailer', 
    yaxis_title='Number of Reviews',
    uniformtext_minsize=8, 
    uniformtext_mode='hide',
    bargap=0.5
)

fig.update_traces(texttemplate='%{text:.2s}')

fig.show()


In [7]:
go.Figure(fig).write_html("eda_plots/iDC_plot3.html", auto_open=True)

Walmart has the most reviews, possibly due to its wide range of brands and products. We’ll explore this more in the next analyses.

### Section 2: Deep Dive into Top Brands:

#### 2.1 Pricing Strategies of Competitors:

In [8]:
filtered_price_df = all_df[all_df['Brand'].isin(top_brands)]

# Remove outliers
filtered_price_df_no_outliers = filtered_price_df.copy()
for brand in top_brands:
    brand_df = filtered_price_df[filtered_price_df['Brand'] == brand]
    Q1 = brand_df['Price'].quantile(0.25)
    Q3 = brand_df['Price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter the brand_df to exclude outliers
    brand_df_no_outliers = brand_df[(brand_df['Price'] >= lower_bound) & (brand_df['Price'] <= upper_bound)]
    filtered_price_df_no_outliers = filtered_price_df_no_outliers[filtered_price_df_no_outliers['Brand'] != brand]  # Exclude the original data for the brand
    filtered_price_df_no_outliers = pd.concat([filtered_price_df_no_outliers, brand_df_no_outliers], axis=0)  # Include the filtered data for the brand


# Initialize visibility list with True for aggregated traces and False for retailer-specific traces
visibility_default = [True]*len(top_brands) + [False]*len(top_brands)*len(file_paths)

# Modify the dropdown buttons
buttons = [
    dict(label="All Retailers", method="update", args=[{"visible": visibility_default}])
]

for i, retailer in enumerate(file_paths.keys()):
    visibility = [False]*len(top_brands) + [False]*i*len(top_brands) + [True]*len(top_brands) + [False]*(len(file_paths)-i-1)*len(top_brands)
    buttons.append(dict(label=retailer, method="update", args=[{"visible": visibility}]))

# Create traces
traces = []

# Additional trace for the aggregated data of all retailers for each brand
for brand in top_brands:
    brand_df = filtered_price_df_no_outliers[filtered_price_df_no_outliers['Brand'] == brand]
    traces.append(go.Box(y=brand_df['Price'], name=f"{brand}", marker_color=px.colors.qualitative.Set3[top_brands.index(brand)], showlegend=True))

# Traces for each brand and retailer
for retailer in file_paths.keys():
    for brand in top_brands:
        brand_retailer_df = filtered_price_df_no_outliers[(filtered_price_df_no_outliers['Brand'] == brand) & (filtered_price_df_no_outliers['Retailer'] == retailer)]
        traces.append(go.Box(y=brand_retailer_df['Price'], name=f"{brand}", marker_color=px.colors.qualitative.Set3[top_brands.index(brand)], showlegend=False))

fig = go.Figure(data=traces)

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(active=0, buttons=buttons)],
    title='Price Distribution of Top 10 TV Brands across Different Retailers (No Outliers)',
    xaxis_title='TV Brand',
    yaxis_title='Price ($)',
    boxmode='group'  
)

# Set default visibility
fig.update_traces(visible=False)
for i in range(len(top_brands)):
    fig.data[i].visible = True

fig.show()

In [9]:
go.Figure(fig).write_html("eda_plots/iDC_plot4.html", auto_open=True)

In [10]:
# Create subplot figure with as many columns as there are top brands
fig = make_subplots(rows=1, cols=len(top_brands), subplot_titles=top_brands, shared_yaxes=True)

# Define unique colors for each retailer
colors = px.colors.qualitative.Set3
if len(file_paths) > len(colors):
    colors = px.colors.qualitative.Alphabet

# Map each retailer to a unique color
color_dict = {retailer: colors[i % len(colors)] for i, retailer in enumerate(file_paths.keys())}

for i, brand in enumerate(top_brands):
    for j, retailer in enumerate(file_paths.keys()):
        # Filter the DataFrame for the specific brand and retailer
        brand_retailer_df = all_df[(all_df['Brand'] == brand) & (all_df['Retailer'] == retailer)]
        
        # Create box plot for each retailer
        fig.add_trace(go.Box(
            y=brand_retailer_df['Price'], 
            name=retailer,
            marker_color=color_dict[retailer],
            boxpoints='outliers',  # Show only outliers beyond the whiskers
            showlegend=(i==0)  # Show legend only for the first subplot to avoid duplication
        ), row=1, col=i+1)

fig.update_layout(
    title='Comparison of Price Distribution of Top Brands across Retailers',
    yaxis_title='Price ($)',
    yaxis_type='log',  # Log transform the y-axis
    boxmode='group'  # Group together boxes of the different traces for each value of x
)

fig.show()

In [11]:
go.Figure(fig).write_html("eda_plots/iDC_plot5.html", auto_open=True)

The price distribution among the top ten brands, based on review numbers, clearly varies. Brands like Samsung, Insignia, and Sony exhibit a broader price range. Additionally, there's noticeable variation in the pricing of the same brand across different retailers. For instance, Walmart generally offers lower prices, potentially reflecting the variety of product lines available, while Bestbuy tends to have higher prices.

#### 2.2 Customer Sentiments and Preferences:

In [12]:
top_brands_df = all_df[all_df['Brand'].isin(top_brands)]
top_brands_df['Brand'].drop_duplicates()

1          Vizio
2            TCL
3        Samsung
6        Sceptre
20            LG
22       Hisense
59         Other
203         Sony
529     Insignia
1048     Toshiba
Name: Brand, dtype: object

In [13]:
fig = px.bar(top_brands_df.groupby(['Brand', 'Retailer'])['Review Mark'].mean().reset_index(), x='Retailer', y='Review Mark', color='Retailer', facet_col='Brand', category_orders={"Brand": top_brands})
fig.show()

In [14]:
go.Figure(fig).write_html("eda_plots/iDC_plot6.html", auto_open=True)

### Section 3: Comprehensive Retailer Evaluation:

#### 3.1 Retailer Competitive Edge:

In [15]:
df = all_df.groupby('Retailer').agg({'Review Mark': 'mean', 'Product Name': 'nunique', 'Review Number': 'sum'}).reset_index()

# Calculate Review Density = Total Number of Reviews / Number of Products
df['Review Density'] = df['Review Number'] / df['Product Name']

# Create figure with secondary y-axis
fig = go.Figure()

# Add bar trace for 'Review Marks'
fig.add_trace(go.Bar(x=df['Retailer'], y=df['Review Mark'], name='Average Review Marks', text=df['Review Mark']))

# Add line trace for 'Review Density', with y-axis on the right side
fig.add_trace(go.Scatter(x=df['Retailer'], y=df['Review Density'], name='Review Density', yaxis='y2'))

# Update layout for dual axis
fig.update_layout(
    title='Retailer Competitive Edge',
    yaxis=dict(title='Average Review Marks'),
    yaxis2=dict(title='Review Density (Number of Reviews per Product)', overlaying='y', side='right')
)

fig.show()

In [16]:
go.Figure(fig).write_html("eda_plots/iDC_plot7.html", auto_open=True)

Bestbuy leads with the highest review marks, contrasting with Walmart, which has the lowest. However, Walmart stands out with the highest review density, indicating a generally higher sales volume. Given our insight that Walmart usually offers lower prices, it seems customers often opt for more affordably priced products even if it might mean compromising on experience quality.

#### 3.2 Strategic Collaboration Insights:

In [17]:
# Calculate the correlation matrix
corr_matrix = all_df[['Price', 'Review Number', 'Review Mark']].corr()

# Create a heatmap
fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.index.tolist(),
    annotation_text=corr_matrix.round(2).values,
    colorscale='Blues',
    showscale=True,
)

fig.update_layout(
    title='Correlation Matrix',
    xaxis=dict(tickangle=-45),
    template='plotly', 
)

fig.show()

In [18]:
go.Figure(fig).write_html("eda_plots/iDC_plot8.html", auto_open=True)

The correlation plot reveals significant relationships among Review Mark, Review Number, and Price. Specifically, Price and Review Number exhibit a negative correlation, suggesting that lower prices can drive higher sales. Similarly, a negative correlation between Price and Review Mark implies that higher prices may often result in lower customer satisfaction.

In [19]:
# Cross-Brand Retailer Analysis
cross_brand_retailer = all_df.groupby(['Retailer', 'Brand']).agg({'Price': 'mean', 'Review Mark': 'mean'}).reset_index()

# Plotting
fig_cross_brand = px.scatter(cross_brand_retailer, x='Price', y='Review Mark', color='Brand', facet_col='Retailer', title='Cross-Brand Retailer Analysis')
fig_cross_brand.show()

In [20]:
go.Figure(fig_cross_brand).write_html("eda_plots/iDC_plot9.html", auto_open=True)

### Conclusions

Through our detailed analysis, several strategic insights have been gleaned to augment iDC TV Inc.'s market stance.

### Strategic Recommendations:

1. Adopt Competitive Pricing:

Implementing competitive pricing can attract a wider consumer base, particularly those sensitive to price, thereby potentially increasing market share.

2. Diversify Product Portfolio:

Enhancing product diversity to cater to various consumer segments can ensure broader market appeal and can help in customer retention through varied offerings.

3. Optimize Market Positioning:

Strategically aligning product offerings and pricing strategies with consumer expectations and preferences can maximize the impact and drive sustained growth.

By strategically aligning with market dynamics and prioritizing consumer-centric approaches, iDC TV Inc. can navigate the competitive landscape effectively and solidify its market presence.